In [2]:
import os

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [3]:
import dspy
dspy.settings.configure(lm=dspy.LM("openai/gpt-4o-mini"))

In [4]:
class QuestionGenerator(dspy.Signature):
    """Generate a yes or no question in order to guess the celebrity name in users' mind. You can ask in general or directly guess the name if you think the signal is enough. You should never ask the same question in the past_questions."""
    past_questions: list[str] = dspy.InputField(desc="past questions asked")
    past_answers: list[bool] = dspy.InputField(desc="past answers")
    new_question: str = dspy.OutputField(desc="new question that can help narrow down the celebrity name")
    guess_mode: bool = dspy.OutputField(desc="If the new_question is the celebrity name guess, set to True, if it is still a general question set to False")

In [5]:
class Reflection(dspy.Signature):
    """Provide reflection on the guessing process"""
    correct_celebrity_name: str = dspy.InputField(desc="the correct celebrity name")
    final_guessor_question: str = dspy.InputField(desc="the final question asked by the guessor")
    past_questions: list[str] = dspy.InputField(desc="past questions asked")
    past_answers: list[bool] = dspy.InputField(desc="past answers")

    reflection: str = dspy.OutputField(
        des="reflection on the guessing process, including what was done well and what can be improved"
    )

In [6]:
def ask(prompt, valid_responses=("y", "n")):
    while True:
        response = input(f"{prompt} ({'/'.join(valid_responses)}): ").strip().lower()
        if response in valid_responses:
            return response
        print(f"Please enter one of: {', '.join(valid_responses)}")

In [7]:
class CelebrityGuess(dspy.Module):
    def __init__(self):
        super().__init__()

        self.question_generator = dspy.ChainOfThought(QuestionGenerator)
        self.reflection = dspy.ChainOfThought(Reflection)

        self.max_retries = 20

    def forward(self):
        celebrity_name = input("What is the name of the celebrity? ")
        past_questions = []
        past_answers = []

        correct_guess = False

        for i in range(self.max_retries):
            question = self.question_generator(
                past_questions=past_questions,
                past_answers=past_answers
            )
            answer = ask(f"{question.new_question}").lower() == "y"
            past_questions.append(question.new_question)
            past_answers.append(answer)

            if question.guess_mode and answer:
                correct_guess = True
                break

            if correct_guess:
                print("Yay! I got it right")
            else:
                print("Opps, I don't guess it right")

            reflection = self.reflection(
                correct_celebrity_name=celebrity_name,
                final_guessor_question=question.new_question,
                past_questions=past_questions,
                past_answers=past_answers
            )
            # print(reflection.reflection)

In [8]:
celeb = CelebrityGuess()
celeb()

Opps, I don't guess it right
Opps, I don't guess it right
Opps, I don't guess it right
Opps, I don't guess it right
Opps, I don't guess it right
Opps, I don't guess it right
Opps, I don't guess it right
Opps, I don't guess it right
Opps, I don't guess it right
Opps, I don't guess it right
Opps, I don't guess it right
Opps, I don't guess it right
